In [1]:
import pandas as pd
from embedding import Embedding
import torch.nn as nn
import torch
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import time

In [2]:
data = pd.read_csv('C:/Users/user/OneDrive/문서/dev/UROP/Crawling/data/sample_comment.csv')
token = pd.read_csv('C:/Users/user/OneDrive/문서/dev/UROP/Crawling/tokenized_0.csv')

In [3]:
df = data[['mbti','comments']]

In [4]:
df

,mbti,comments
0,ENTJ,눈빛과 기류를 보면 알지 않나용 일단 진심으로 이뻐하는 게 여기까지 느껴지네요
1,ENFJ,저도 눈빛과 기류를 보고 알수있다고 생각해요
2,ENFP,걱정하지마세요 엔프제라고 해서 다 조리있게 말하지는 않더라구요 대신 공통점이 칭찬은...
3,ENFP,네 말도 많구 말에 칭찬이 많이 차지하더라구요
4,ISTP,밀당아닌가요
...,...,...
995,INFJ,확실한건 너무좋아하면 저처럼 읽씹당해도 또보내긴 하는데요 저런 잘자라는 멘트는 좋아...
996,ISTP,후전 연락온다면 적극적으로 다시 변할것 같은데굳이 저혼자 자꾸 먼저 보내기가 싫네요...
997,INFJ,망상의동물이라 시간이 지날수록 혼자 망상하다가 마음정리할수도 있어요 지금 비유를하면...
998,ISTP,카톡 프사에도 반응하나요 제 얼굴 사진으로 바꿔볼까해서요


In [5]:
df = df.dropna().reset_index(drop=True)
token = token.dropna().reset_index(drop=True)

In [6]:
df

,mbti,comments
0,ENTJ,눈빛과 기류를 보면 알지 않나용 일단 진심으로 이뻐하는 게 여기까지 느껴지네요
1,ENFJ,저도 눈빛과 기류를 보고 알수있다고 생각해요
2,ENFP,걱정하지마세요 엔프제라고 해서 다 조리있게 말하지는 않더라구요 대신 공통점이 칭찬은...
3,ENFP,네 말도 많구 말에 칭찬이 많이 차지하더라구요
4,ISTP,밀당아닌가요
...,...,...
982,INFJ,확실한건 너무좋아하면 저처럼 읽씹당해도 또보내긴 하는데요 저런 잘자라는 멘트는 좋아...
983,ISTP,후전 연락온다면 적극적으로 다시 변할것 같은데굳이 저혼자 자꾸 먼저 보내기가 싫네요...
984,INFJ,망상의동물이라 시간이 지날수록 혼자 망상하다가 마음정리할수도 있어요 지금 비유를하면...
985,ISTP,카톡 프사에도 반응하나요 제 얼굴 사진으로 바꿔볼까해서요


In [7]:
for i in range(len(df)):
    if 'E' in df.loc[i, 'mbti']:
        df.loc[i, 'mbti'] = 0
    else:
        df.loc[i, 'mbti'] = 1

In [8]:
label = [int(i) for i in df['mbti'].values.tolist()]
print(len(label))
label_tensor = torch.LongTensor(label)
print(label_tensor)

987
tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 1, 1

In [9]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.hidden = self.init_hidden(batch_size)

    def init_hidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))

    def forward(self, input):
        batch_size = input.size(0)
        self.hidden = self.init_hidden(batch_size)
        lstm_out, self.hidden = self.lstm(input, self.hidden)
        output = self.fc(lstm_out[:, -1, :])
        return output

In [10]:
em = Embedding(sereis=token['comments'])
word2vec_model = em.get_embedding_model()

[I 2023-12-10 04:00:43,587] A new study created in memory with name: no-name-5be9b333-263b-4041-a739-242a1b9c60dd


[I 2023-12-10 04:00:43,652] Trial 0 finished with value: 90942.84375 and parameters: {'vector_size': 18, 'window': 3, 'min_count': 5, 'sg': 0}. Best is trial 0 with value: 90942.84375.
[I 2023-12-10 04:00:43,693] Trial 1 finished with value: 28680.904296875 and parameters: {'vector_size': 39, 'window': 9, 'min_count': 29, 'sg': 0}. Best is trial 1 with value: 28680.904296875.
[I 2023-12-10 04:00:43,733] Trial 2 finished with value: 30472.04296875 and parameters: {'vector_size': 19, 'window': 3, 'min_count': 28, 'sg': 0}. Best is trial 1 with value: 28680.904296875.
[I 2023-12-10 04:00:43,781] Trial 3 finished with value: 60951.8125 and parameters: {'vector_size': 24, 'window': 3, 'min_count': 10, 'sg': 0}. Best is trial 1 with value: 28680.904296875.
[I 2023-12-10 04:00:43,839] Trial 4 finished with value: 68820.8671875 and parameters: {'vector_size': 32, 'window': 10, 'min_count': 7, 'sg': 0}. Best is trial 1 with value: 28680.904296875.
[I 2023-12-10 04:00:43,893] Trial 5 finished wi

In [11]:
# Word2Vec 모델의 크기를 기반으로 LSTM 모델 생성
input_size = word2vec_model.vector_size
hidden_size = 128
output_size = 1
batch_size = 32
# vocab_size = len(word2vec_model.wv.key_to_index)
lstm_model = LSTMModel(input_size, hidden_size, output_size, batch_size)
# print(vocab_size)

In [12]:
# 단어에 대한 0 벡터 생성
def get_word_vector(word, model):
    try:
        return model.wv[word]
    except:
        return np.zeros(input_size)

# 모든 단어에 대한 벡터 리스트 생성
# word_vectors = [get_word_vector(word, word2vec_model) for sentence in em.corpus for word in sentence]
end = 100
word_vectors = []

for sentence in em.corpus:
    temp = []
    for word in sentence[:end]:  # 문장을 end 길이로 자르기
        temp.append(get_word_vector(word, word2vec_model))

    # 패딩 추가
    padding_length = end - len(temp)
    if padding_length > 0:
        padding = [np.zeros(word2vec_model.vector_size) for _ in range(padding_length)]
        temp.extend(padding)

    word_vectors.append(temp)

In [14]:
# 벡터 리스트를 텐서로 변환
input_tensor = torch.tensor(word_vectors, dtype=torch.float32)
# input_tensor = torch.LongTensor(word_vectors)

C:\Users\user\AppData\Local\Temp\ipykernel_24136\3932551150.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  input_tensor = torch.tensor(word_vectors, dtype=torch.float32)


In [15]:
trn_data=input_tensor[:800]
trn_label=label_tensor[:800]
val_data=input_tensor[800:]
val_label=label_tensor[800:]

In [16]:
print(trn_data.shape)
print(trn_label.shape)
print(val_data.shape)
print(val_label.shape)

torch.Size([800, 100, 25])
torch.Size([800])
torch.Size([187, 100, 25])
torch.Size([187])


In [17]:
# len(input_tensor[0])
print(input_tensor.shape)
print(input_tensor.size(-1))

torch.Size([987, 100, 25])
25


- LSTM 모델에 입력 전달하여 출력 받기
output = lstm_model(input_tensor)
print("LSTM 모델의 출력:", output, len(output))

In [18]:
# BCE 손실 함수 정의
criterion = nn.BCEWithLogitsLoss()
# Adam 옵티마이저 설정
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

In [19]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [20]:
# data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# dataset = MyDataset(input_tensor, label_tensor)
trn_dataset = MyDataset(trn_data, trn_label)
val_dataset = MyDataset(val_data, val_label)

# batch_size = 32
train_loader = DataLoader (trn_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader (val_dataset, batch_size=batch_size, shuffle=True)

In [21]:
# 학습 및 평가 루프 정의
def train(model, criterion, optimizer, train_loader, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:  
            # train_loader에는 입력 데이터와 레이블이 있는 데이터가 포함되어야 합니다.
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())  
            # BCELoss를 사용하므로 레이블을 float로 변환합니다.
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

In [22]:
def evaluate(model, criterion, eval_loader):
    model.eval()  # 평가 모드로 모델 설정
    total_loss = 0.0
    total_corrects = 0
    with torch.no_grad():  # 기울기 계산 비활성화
        for inputs, labels in eval_loader:
            # if inputs.size(0) == 1:
            # print(inputs.shape)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())

            total_loss += loss.item()
            # 예측된 클래스를 계산하고, 정확도를 측정합니다.
            # 이 부분은 사용하는 모델과 문제에 따라 다를 수 있습니다.
            preds = outputs.round()  # 예: 이진 분류의 경우
            total_corrects += torch.sum(preds == labels.data)

    # 평균 손실과 정확도 계산
    avg_loss = total_loss / len(eval_loader)
    accuracy = total_corrects.double() / len(eval_loader.dataset)

    print(f"Loss: {avg_loss}, Accuracy: {accuracy}")

    return avg_loss, accuracy

In [23]:
def train_and_evaluate(model, criterion, optimizer, train_loader, eval_loader, epochs=5):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")

        # 학습 루프
        train(model, criterion, optimizer, train_loader, epochs=1)

        # 평가 루프
        eval_loss, eval_accuracy = evaluate(model, criterion, eval_loader)

        # 에포크 결과 출력
        print(f"Validation Loss: {eval_loss}, Validation Accuracy: {eval_accuracy}")

# 모델, 손실 함수, 옵티마이저, 데이터 로더 등을 정의한 후
train_and_evaluate(lstm_model, criterion, optimizer, train_loader, val_loader, epochs=10)


Epoch 1/5


Epoch 1/1, Loss: 0.6167220044136047
Loss: 0.4659629414478938, Accuracy: 27.764705882352942
Validation Loss: 0.4659629414478938, Validation Accuracy: 27.764705882352942
Epoch 2/5
Epoch 1/1, Loss: 0.5574532663822174
Loss: 0.39106598993142444, Accuracy: 27.737967914438503
Validation Loss: 0.39106598993142444, Validation Accuracy: 27.737967914438503
Epoch 3/5
Epoch 1/1, Loss: 0.5553237903118133
Loss: 0.41958943506081897, Accuracy: 27.711229946524064
Validation Loss: 0.41958943506081897, Validation Accuracy: 27.711229946524064
Epoch 4/5
Epoch 1/1, Loss: 0.5543683660030365
Loss: 0.39215486248334247, Accuracy: 27.79144385026738
Validation Loss: 0.39215486248334247, Validation Accuracy: 27.79144385026738
Epoch 5/5
Epoch 1/1, Loss: 0.5603959405422211
Loss: 0.4086423267920812, Accuracy: 27.764705882352942
Validation Loss: 0.4086423267920812, Validation Accuracy: 27.764705882352942
